# Training

In this section you can train the LSTM model with Bach. The first section is monophony and the second with polyphony encoding.

## Train monophony encoding

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

We use the music21 corpus for the bach training data. </br>
You can adjust the `paths` to reduce the number of training songs. </br>
</br>
See: https://web.mit.edu/music21/doc/about/referenceCorpus.html

To use custom training data use:
```python
from pathlib import Path

paths = list(Path("./dataset/Pokemon").rglob("*.midi"))
encoded_songs = utils.load_dataset_lstm(paths, 64, mapping, corpus=False)
```

In [ ]:
paths = m21.corpus.getComposer('bach')
# paths = paths[:10]

print(f"Found {len(paths)} songs in corpus.")
encoded_songs = utils.load_dataset_lstm(paths, 64, mapping)

In [ ]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

# For tensorboard logging
logdir = "logs/bach_lstm"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, epochs=50, tensorboard_callback=tensorboard_callback)

To load the pre trainend model you have to save the model. It will be saved at `mukkeBude/model/preTrainedModels`

In [ ]:
model.save("Bach_soloMelodie_lstm")

In [ ]:
# Show the logs
%tensorboard --logdir logs/bach_lstm

## Train polyphony encoding

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

To load custom training data use:
```python
# Load songs
songs = utils.read_all("./dataset/Pokemon/")

encoded_songs = []
for song in songs:
    song = utils.read_single(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.append(mapping.textify(encoded_song))
```

In [ ]:
# Load songs
paths = m21.corpus.getComposer('bach')
# paths = paths[:10]

print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single_from_corpus(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.extend(encoded_song)

In [ ]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

logdir = "logs/bach_lstm_polyphonie"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, epochs=50, tensorboard_callback=tensorboard_callback)

In [ ]:
# Save model
model.save("Bach_polyphonie_lstm")

In [ ]:
# Show the logs
%tensorboard --logdir logs/bach_lstm_polyphonie

# Generate music

In this section you can generate music with a pre trained LSTM model. The first section is monophony and the second with polyphony encoding.

## Generate monophony

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import REST
from mukkeBude.mapping import WAIT_LSTM


# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

In [ ]:
model = MukkeBudeLSTM.load(mapping, "Bach_soloMelodie_lstm")

You need to pass a start seed. You can also give the network a max length for the generation (it can be shorter but not longer)

In [ ]:
# Create song
generated_song = model.generate("n72 _ _ _ _ _ n72 _ _ _ _ _ n72 _ n71 _", max_length=500)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(REST)
special_tokens.remove(WAIT_LSTM)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), WAIT_LSTM, special_tokens))

In [ ]:
new_song = utils.decode_songs_old(generated_song)
print(generated_song)

In [ ]:
path = Path("generated_song_bach_lstm_mono.midi")
utils.write_midi(new_song, path)

To open the midi file we recomend [musecore](https://musescore.org/de)

## Generate polyphony

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import SEP
from mukkeBude.mapping import BOS


# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

In [ ]:
model = MukkeBudeLSTM.load(mapping, "Bach_polyphonie_lstm")

In [ ]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep", 500)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(SEP)
special_tokens.remove(BOS)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), "d1", special_tokens))

In [ ]:
new_song_ints = mapping.numericalize(generated_song.split(" "))
new_song = utils.from_polyphonic_encoding(np.array(new_song_ints), mapping, bpm=100)

print(generated_song)

In [ ]:
path = Path("generated_song_bach_lstm_poly.mid")
utils.write_midi(new_song, path)